# Get data via SQLAlchemy

In [ ]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker
from sqlalchemy import and_, or_
import MySQLdb, pickle

In [ ]:
import pandas as pd

### Access to MySQL Server

In [ ]:
pw = pickle.load(open("../crawler/mysql_pw.pickle", "rb"))

In [ ]:
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + pw + "@52.78.44.120/project_rookie")

### ORM

In [ ]:
Base = declarative_base()

In [ ]:
def disp(datas):
    for data in datas:
        print(data)

In [ ]:
def orm_to_df(orm):

    df = pd.DataFrame(columns=['artist', 'album','freq_billboard'])

    for i in orm:

        data = {
            'artist' : i.__dict__['artist'],
            'album' : i.__dict__['album'],
            'freq_billboard' : i.__dict__['freq_billboard'],
        }
        df.loc[len(df)] = data
        
    return df

### ORM - Debut Album

In [ ]:
class Debut(Base):
    __tablename__ = 'debut_album'

    index = Column(Integer, primary_key=True)
    artist = Column(String)
    album = Column(String)
    genre = Column(String)
    single_count = Column(Integer)
    
    def __init__(self, index, artist, album, genre, single_count):
        self.index = index
        self.artist = artist
        self.album = album
        self.genre = genre
        self.single_count = single_count
    
    def __repr__(self):
        return "<index {}, {}, {}, {}, {}>".format(self.index, self.artist, self.album, self.genre, self.single_count)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
results = session.query(Debut).all()
results[0].__dict__

In [ ]:
hiphop = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["hop"])))
hiphop_df = orm_to_df(hiphop)
print(hiphop_df.shape)
hiphop_df.head()

In [ ]:
rap = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["Rap"])))
rap_df = orm_to_df(rap)
print(rap_df.shape)
rap_df.head()

In [ ]:
trap = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["trap"])))
trap_df = orm_to_df(trap)
print(trap_df.shape)
trap_df.head()

In [ ]:
hiphop_df = pd.concat([hiphop_df, rap_df, trap_df])
hiphop_df.drop_duplicates(['artist', 'album'], inplace=True)
hiphop_df['genre'] = "hiphop"
hiphop_df

In [ ]:
rnb = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["R&B"])))
rnb_df = orm_to_df(rnb)
rnb_df['genre'] = 'rnb'
print(rnb_df.shape)
rnb_df.head()

In [ ]:
soul = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["soul"])))
soul_df = orm_to_df(soul)
soul_df['genre'] = 'soul'
print(soul_df.shape)
soul_df.head()

In [ ]:
funk = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["funk"])))
funk_df = orm_to_df(funk)
funk_df['genre'] = 'funk'
print(funk_df.shape)
funk_df.head()

In [ ]:
pop = session.query(Debut).filter(and_(~Debut.genre.contains(['K-pop']), Debut.genre.contains(["pop"])))
pop_df = orm_to_df(pop)
pop_df['genre'] = 'pop'
print(pop_df.shape)
pop_df.head()

### Concat all genres and drop duplicates rows (overlapping genres)

In [ ]:
df_list = pd.concat([hiphop_df, rap_df, trap_df, rnb_df, soul_df, funk_df, pop_df])
len(df_list)

In [ ]:
df_list = df_list.drop_duplicates(['artist', 'album'])
print(df_list.shape)
df_list.head()

### Save to csv file

In [ ]:
df_list.to_csv('../data/final_list_debut_album.csv', index=False)